<a href="https://colab.research.google.com/github/hardeybisey/neural-network/blob/main/traingpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tokenizers
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.0 MB/s eta 0:00:00
--2023-05-10 13:54:30--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-05-10 13:54:31 (21.9 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from tokenizers import Tokenizer,PreTokenizedString,NormalizedString, pre_tokenizers, CharBPETokenizer , decoders,  normalizers,ByteLevelBPETokenizer
from tokenizers.models import BPE , WordPiece
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace, PreTokenizer
from tokenizers.processors import TemplateProcessing
from tokenizers.normalizers import NFD, Lowercase, StripAccents
torch.manual_seed(45562)

# Read the dataset

In [3]:
text = open('input.txt', 'r', encoding='utf-8').read()
print("length of dataset in characters: ", len(text))
print(f"\n{text[:200]}")

length of dataset in characters:  1115394

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


# Build a custom tokenizer with hugging face  tokenizer API

In [11]:
sp_tokenizer = Tokenizer(BPE(unk_token="<unk>"))
sp_tokenizer.normalizer = normalizers.Sequence([NFD(), StripAccents()])
# sp_tokenizer.pre_tokenizer =
# sp_tokenizer.decoder = decoders.BPEDecoder(suffix="</w>")
# sp_tokenizer.post_processor = TemplateProcessing(single="[CLS] $A [SEP]",
#                                                  pair="[CLS] $A [SEP] $B:1 [SEP]:1",
#                                                  special_tokens=[
#                                                     ("[CLS]", 1),
#                                                     ("[SEP]", 2)])

# trainer = BpeTrainer(vocab_size=500, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
# sp_tokenizer.train_from_iterator(text , trainer)

In [12]:
print(sp_tokenizer.encode(text[:50]).tokens)
print(sp_tokenizer.encode(text[:50]).ids)

Exception: ignored

In [75]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
tokenizer.normalizer = normalizers.Sequence([normalizers.NFD(), normalizers.StripAccents()])
tokenizer.pre_tokenizer = 
# tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]", ]
trainer = trainers.WordPieceTrainer(vocab_size=500, special_tokens=special_tokens)
tokenizer.train_from_iterator(text, trainer=trainer)
# pre_tokenizer.pre_tokenize_str(text[:50])



from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
from datasets import load_dataset


tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)
tokenizer.pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Split(" ","merged_with_next"), pre_tokenizers.Punctuation()])
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)



In [58]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [59]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", 2), ("[SEP]", 3)],
)

In [76]:
tokenizer.pre_tokenizer.pre_tokenize_str(text[:50])

[('First', (0, 5)),
 (' Citizen', (5, 13)),
 (':', (13, 14)),
 ('\nBefore', (14, 21)),
 (' we', (21, 24)),
 (' proceed', (24, 32)),
 (' any', (32, 36)),
 (' further', (36, 44)),
 (',', (44, 45)),
 (' hear', (45, 50))]

In [78]:
encoding = tokenizer.encode("Let's test this tokenizer...", "on a pair of sentences.")
print(encoding.tokens)
print(encoding.type_ids)

['[UNK]', "'", 's', '[UNK]', '[UNK]', '[UNK]', '.', '.', '.', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '.']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]


In [79]:
print(tokenizer.encode(text[:500]).tokens)

['[UNK]', '[UNK]', ':', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', ',', '[UNK]', '[UNK]', '[UNK]', '.', '[UNK]', ':', '[UNK]', ',', '[UNK]', '.', '[UNK]', '[UNK]', ':', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '?', '[UNK]', ':', '[UNK]', '.', '[UNK]', '.', '[UNK]', '[UNK]', ':', '[UNK]', ',', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '.', '[UNK]', ':', '[UNK]', '[UNK]', "'", 't', ',', '[UNK]', '[UNK]', "'", 't', '.', '[UNK]', '[UNK]', ':', '[UNK]', '[UNK]', '[UNK]', '[UNK]', ',', '[UNK]', '[UNK]', "'", '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '.', '[UNK]', "'", 't', '[UNK]', '[UNK]', '?', '[UNK]', ':', '[UNK]', '[UNK]', '[UNK]', '[UNK]', "'", 't', ';', '[UNK]', '[UNK]', '[UNK]', '[UNK]', ':', '[UNK]', ',', '[UNK]', '!', '[UNK]', '[UNK]', ':', '[UNK]', '[UNK]', ',', '[UNK]', '[UNK]', '.', '[UNK]', '[UNK]', ':', '[UNK]', '[UNK]', '[UNK]', '[UNK]']


In [ ]:
tokenizer = Tokenizer.from_pretrained("bert-base-uncased")

In [41]:
tokenizer.save("w.json")

In [ ]:
tokenizer = Tokenizer(BPE())
# tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
# tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
# tokenizer.pre_tokenizer.pre_tokenize_str(text[:50])
trainer = BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>","<n>"])
tokenizer.train_from_iterator(text, trainer=trainer)

In [ ]:
# from tokenizers import CharBPETokenizer
# sp_tokenizer = CharBPETokenizer()
# sp_tokenizer.train_from_iterator(text)
sp_tokenizer.add_special_tokens(["\n", '[SEP]', '[PAD]', '[CLS]'])
sp_tokenizer.post_processor = TemplateProcessing(special_tokens=[ (0, "<unk>"), (127,"\n")])
sp_tokenizer.decoder = decoders.BPEDecoder()
vocab_size = sp_tokenizer.get_vocab_size()

In [ ]:
print(sp_tokenizer.decode([17, 127, 55, 56, 78, 14, 46, 57, 46, 63, 42, 96, 100, 127, 13, 42, 43, 52, 55,127]))

Frst Citizen : Befor


In [ ]:
encoding = sp_tokenizer.encode(text[:50])
print(encoding.tokens)
print(encoding.ids)
print(encoding.ids)

['F', 'i', 'r', 's', 't</w>', 'C', 'i', 't', 'i', 'z', 'e', 'n</w>', ':</w>', '\n', 'B', 'e', 'f', 'o', 'r', 'e</w>', 'w', 'e</w>', 'p', 'r', 'o', 'c', 'e', 'e', 'd</w>', 'a', 'n', 'y</w>', 'f', 'u', 'r', 't', 'h', 'e', 'r</w>', ',</w>', 'h', 'e', 'a', 'r</w>']
[17, 46, 55, 56, 78, 14, 46, 57, 46, 63, 42, 96, 100, 127, 13, 42, 43, 52, 55, 114, 60, 114, 53, 55, 52, 40, 42, 42, 101, 38, 51, 111, 43, 58, 55, 57, 45, 42, 104, 99, 45, 42, 38, 104]
[17, 46, 55, 56, 78, 14, 46, 57, 46, 63, 42, 96, 100, 127, 13, 42, 43, 52, 55, 114, 60, 114, 53, 55, 52, 40, 42, 42, 101, 38, 51, 111, 43, 58, 55, 57, 45, 42, 104, 99, 45, 42, 38, 104]


In [ ]:
vocab_size

131

In [ ]:
sp_tokenizer.token_to_id("\n")

127

In [85]:
from datasets import load_dataset

load_dataset("wikipedia", "20220301.en")

Downloading:   0%|          | 0.00/15.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/20.3G [00:00<?, ?B/s]

KeyboardInterrupt: ignored

In [ ]:
# worked
# tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
# tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)
# tokenizer.normalizer = normalizers.Sequence(
#     [normalizers.NFD(), normalizers.StripAccents()])
# tokenizer.pre_tokenizer = pre_tokenizers.Sequence([pre_tokenizers.Split('', 'isolated'),
#                                                    pre_tokenizers.Punctuation()])

# tokenizer.pre_tokenizer.pre_tokenize_str("Let's test my \n pre-tokenizer.")
# special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
# trainer = trainers.WordPieceTrainer(vocab_size=5000, special_tokens=special_tokens)
# tokenizer.train_from_iterator(text, trainer=trainer)

In [ ]:
print(tokenizer.encode(text[:50]).ids) 
print(tokenizer.encode(text[:50]).tokens)

[23, 52, 61, 62, 63, 6, 20, 52, 63, 52, 69, 48, 57, 15, 5, 19, 48, 49, 58, 61, 48, 6, 66, 48, 6, 59, 61, 58, 46, 48, 48, 47, 6, 44, 57, 68, 6, 49, 64, 61, 63, 51, 48, 61, 11, 6, 51, 48, 44, 61]
['F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i', 'z', 'e', 'n', ':', '\n', 'B', 'e', 'f', 'o', 'r', 'e', ' ', 'w', 'e', ' ', 'p', 'r', 'o', 'c', 'e', 'e', 'd', ' ', 'a', 'n', 'y', ' ', 'f', 'u', 'r', 't', 'h', 'e', 'r', ',', ' ', 'h', 'e', 'a', 'r']


In [ ]:


dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")

def get_training_corpus(text):
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"] + text

In [83]:
encoding = tokenizer.encode(text[:500])
print(encoding.tokens)

['first', 'citizen', ':', 'before', 'we', 'proceed', 'any', 'further', ',', 'hear', 'me', 'speak', '.', 'all', ':', 'speak', ',', 'speak', '.', 'first', 'citizen', ':', 'you', 'are', 'all', 'resolved', 'rather', 'to', 'die', 'than', 'to', 'fam', '##ish', '?', 'all', ':', 'resolved', '.', 'resolved', '.', 'first', 'citizen', ':', 'first', ',', 'you', 'know', 'cai', '##us', 'marc', '##ius', 'is', 'chief', 'enemy', 'to', 'the', 'people', '.', 'all', ':', 'we', 'know', "'", 't', ',', 'we', 'know', "'", 't', '.', 'first', 'citizen', ':', 'let', 'us', 'kill', 'him', ',', 'and', 'we', "'", 'll', 'have', 'corn', 'at', 'our', 'own', 'price', '.', 'is', "'", 't', 'a', 'verdict', '?', 'all', ':', 'no', 'more', 'talking', 'on', "'", 't', ';', 'let', 'it', 'be', 'done', ':', 'away', ',', 'away', '!', 'second', 'citizen', ':', 'one', 'word', ',', 'good', 'citizens', '.', 'first', 'citizen', ':', 'we', 'are', 'accounted', 'poor']
